In [0]:
import pandas as pd
import numpy as np
import time

### TL;DR

Vectorize. Use np.where() or np.select() over all other options. If you need to use Pandas go for df.loc. Use df.apply() if there no other way to get it done, and never ever touch df.iterrrows()

In [0]:
# Creating the dataset
data = np.random.randint(5, size=(100_000, 4))
df = pd.DataFrame(data=data, columns=['a', 'b', 'c', 'd'])
print(df.shape)
df.head(3)
#df.to_csv(f"/dbfs/mnt/g2_workspaces/nico/rtc/flights/processed/ext_table.csv", index=False)

(100000, 4)
Out[113]:

,a,b,c,d
0,0,4,2,0
1,2,3,2,2
2,0,1,1,2


In [0]:
def delete_column(df, col):
    try:
        df.drop(columns=col, inplace=True)
    except:
        pass

In [0]:
## Function + Apply.

def func(row):
    if row['a'] > row['b']:
        return 'Right'
    elif row['a'] < row['b']:
        return 'Left'
    else:
        return 'Same'
    
delete_column(df, 'result')    

t1 = time.perf_counter()
df['result'] = df.apply(func, axis=1)
print(time.perf_counter() - t1)
df.head()

1.5958310360001633
Out[115]:

,a,b,c,d,result
0,0,4,2,0,Left
1,2,3,2,2,Left
2,0,1,1,2,Left
3,2,1,1,3,Right
4,1,0,3,0,Right


In [0]:
## Lambda Function + Apply.

delete_column(df, 'result')

t1 = time.perf_counter()
df['result'] = df.apply(lambda x: 'Right' if x['a'] > x['b'] else ('Left' if x['a'] < x['b'] else 'Same'), axis=1)
print(time.perf_counter() - t1)
df.head()

1.6283594300002733
Out[116]:

,a,b,c,d,result
0,0,4,2,0,Left
1,2,3,2,2,Left
2,0,1,1,2,Left
3,2,1,1,3,Right
4,1,0,3,0,Right


In [0]:
## Vectorizing with loc.
## [if else condition, new column name] = value if condition is met

delete_column(df, 'result')

t1 = time.perf_counter()
df.loc[df['a'] > df['b'], 'result'] = 'Right'
df.loc[df['a'] < df['b'], 'result'] = 'Left'
df.loc[df['a'] == df['b'], 'result'] = 'Same'
print(time.perf_counter() - t1)
df.head()

0.01928263700028765
Out[117]:

,a,b,c,d,result
0,0,4,2,0,Left
1,2,3,2,2,Left
2,0,1,1,2,Left
3,2,1,1,3,Right
4,1,0,3,0,Right


In [0]:
## NumPy where()
## np.where(if else condition, value if true, value if false)

delete_column(df, 'result')

t1 = time.perf_counter()
df['result'] = np.where(df['a'] > df['b'], 'Right', np.where(df['a'] < df['b'], 'Left', 'Same'))
print(time.perf_counter() - t1)
df.head()

0.021189668001170503
Out[118]:

,a,b,c,d,result
0,0,4,2,0,Left
1,2,3,2,2,Left
2,0,1,1,2,Left
3,2,1,1,3,Right
4,1,0,3,0,Right


In [0]:
## NumPy select()
##

delete_column(df, 'result') 

t1 = time.perf_counter()
condition_list = [df['a'] > df['b'], df['a'] < df['b'], df['a'] == df['b']]
condition_met = ['Right', 'Left', 'Same']
df['result'] = np.select(condlist=condition_list, choicelist=condition_met, default=0)
print(time.perf_counter() - t1)
df.head()

0.0339097279975249
Out[119]:

,a,b,c,d,result
0,0,4,2,0,Left
1,2,3,2,2,Left
2,0,1,1,2,Left
3,2,1,1,3,Right
4,1,0,3,0,Right


In [0]:
## Pandas iterrows()/itertuples()
## iterrows() is immediately ruled out since it has a worse performance than itertuples()
## Never use this iterrrows. It is the worst of all.

delete_column(df, 'result') 

t1 = time.perf_counter()
df['result'] = ['Right' if row.a > row.b else ('Left' if row.a < row.b else 'Same') for row in df.itertuples()]
print(time.perf_counter() - t1)
df.head()

0.0957714100004523
Out[120]:

,a,b,c,d,result
0,0,4,2,0,Left
1,2,3,2,2,Left
2,0,1,1,2,Left
3,2,1,1,3,Right
4,1,0,3,0,Right


Happy coding :)